# GPU를 사용한 PyTorch 모델 학습

이 노트북에서는 GPU 컴퓨팅 클러스터를 사용하여 MNIST 데이터에 대해 PyTorch 합성곱 신경망(CNN) 모델을 학습시킵니다.

> 이 노트북은 PyTorch를 사용하는 Azure Machine Learning 예제를 기반으로 합니다: https://github.com/Azure/azureml-examples/tree/main/python-sdk/workflows/train/pytorch/mnist

In [ ]:
# imports

from azureml.core import Workspace
from azureml.core import ScriptRunConfig, Experiment, Environment

## 환경 변수 가져오기

작업을 제출하기 전에 작업 공간, 환경, 컴퓨팅 및 입력 인자와 같은 필요한 모든 환경 변수를 확보해야 합니다.

In [ ]:
import os
import glob
from azureml.core import Workspace

try:
    _ = os.getcwd()
except FileNotFoundError:
    os.chdir(os.path.expanduser("~"))

ws = None
candidate_paths = [os.getcwd(), os.path.expanduser("~/cloudfiles/code/Users")]
candidate_paths.extend(glob.glob(os.path.expanduser("~/cloudfiles/code/Users/*")))

for candidate_path in candidate_paths:
    try:
        ws = Workspace.from_config(path=candidate_path)
        break
    except Exception:
        pass

if ws is None:
    try:
        ws = Workspace.get(
            name="aml-dl-workshop",
            resource_group="rg-aml-dl-workshop",
            subscription_id=os.environ.get("AZUREML_ARM_SUBSCRIPTION")
        )
    except Exception:
        ws = Workspace.from_config()

print(f"Loaded workspace: {ws.name}")

In [ ]:
from azureml.core import Environment

pytorch_env = Environment.get(workspace=ws, name="AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu")

## 구성 정의 및 실행 제출

필요한 변수를 모두 정의했으므로 이제 스크립트 실행 구성을 정의하고 실행을 제출할 수 있습니다.

`00-setup.md`와 `01-preprocess-data-rapids.md`를 따라 오셨다면 compute target은 `cc-aml-gpu`를 사용합니다.

In [ ]:
import os
import glob
from azureml.core import ScriptRunConfig

script_dir = None
candidate_dirs = [
    os.path.join(os.getcwd(), "script"),
    os.path.expanduser("~/cloudfiles/code/Users/*/azure-machine-learning-workshop/Notebooks/02-train-model/script"),
    os.path.expanduser("~/cloudfiles/code/Users/*/Notebooks/02-train-model/script")
]

for candidate in candidate_dirs:
    if "*" in candidate:
        matches = glob.glob(candidate)
        if matches:
            script_dir = matches[0]
            break
    elif os.path.isdir(candidate):
        script_dir = candidate
        break

if script_dir is None:
    raise FileNotFoundError("script directory not found. Please check notebook repo path.")

print(f"Using source_directory: {script_dir}")

src = ScriptRunConfig(source_directory=script_dir,
                      script='train.py',
                      arguments=["--epochs", 2],
                      compute_target="cc-aml-gpu",
                      environment=pytorch_env)

학습 과정에서 어떤 작업이 수행되는지 알아보려면 `script` 폴더에 있는 `train.py` 스크립트를 살펴보세요.

다음 셀은 스크립트 실행을 시작합니다. 먼저 컴퓨팅 클러스터의 노드 수가 0개에서 확장되어야 합니다. 노드가 사용 가능해지면 스크립트가 실행됩니다. 스크립트 실행은 빠르며, 실행 시간은 실행 후 **Experiment** 보고서의 **Details** 탭에서 확인할 수 있습니다.

In [ ]:
from azureml.core import Experiment

run = Experiment(ws,'train-model').submit(src)
run.wait_for_completion(show_output=True)

스튜디오에서 새 실행이 시작되어 진행 중이라는 알림을 받게 됩니다.

또는 **Experiments** 탭으로 이동하여 `train-model` 실험을 찾을 수도 있습니다.

실행이 완료되면 **Metrics** 탭에서 두 에포크에 대한 두 가지 평가 지표를 확인할 수 있습니다. **Details** 탭에서는 실행에 소요된 시간을 볼 수 있습니다.